<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

<h2 style="color: #FFA07A;">2. Explorar dados</h2>

In [ ]:
#
import ipywidgets as widgets
import time

# --- Criar um aplicativo em HTML para exibir o efeito de escrita ---
output = widgets.HTML(value="<div></div>")
display(output)

# --- Texto formatado para o efeito de escrita ---
texto = """
<h3>Como preparámos os dados?</h3> <p><b>Os dados foram pré-processados e estruturados em ficheiros no formato .csv através dos seguintes procedimentos:</b></p> 
<p>🔸 <b>.csv categoria_servicos:</b> contém edifícios do município do Porto extraídos do OpenStreetMap (OSM) e serviços essenciais identificados no Google Maps. Foram excluídos edifícios não residenciais. A cada edifício foram associados os serviços num raio de 1.5 km (15 minutos a pé). Utilizou-se o algoritmo <b>K-D Tree</b> para consultas de proximidade e o algoritmo <b>Dijkstra</b> para encontrar o caminho mais curto entre dois pontos num grafo.</p> 
<p>🔸 <b>.csv distancia_servicos:</b> inclui os procedimentos anteriores, e acrescenta o cálculo da distância média aos serviços, considerando percursos a pé até 1.5 km. Cada edifício tem associada uma distância média aos serviços disponíveis.</p> 
<p>🔸 <b>.csv pop+65:</b> distribui a população com 65 anos ou mais pelos edifícios de cada unidade da Base Geográfica de Referenciação de Informação (BGRI), proporcional à área dos edifícios e à população de cada unidade.</p> <p><b>Na investigação, foram considerados 31.873 edifícios, com 58.748 residentes com 65 anos ou mais. Foram identificados os seguintes serviços:</b>
</p> <ul> <li>🔸 Bancos (100)</li> <li>🔸 Supermercados (94)</li> <li>🔸 Farmácias (77)</li> <li>🔸 Parques ou jardins (33)</li> <li>🔸 CTT (24)</li> <li>🔸 Centros de saúde (18)</li> <li>🔸 Hospitais (2)</li> </ul> 
<div> 🔸 <b><u>A unidade de análise de todo o estudo é o edifício.</b></u> Para mais detalhes sobre os processos de pré-processamento dos dados, consulte a documentação completa no <b><a href="https://github.com/RobertoOlivetree/Average_Distance_to_Services_by_Category.git" target="_blank">GitHub</a></b>. </div> 
</div> 
"""

# --- Criar o efeito de escrita dentro do HTML mantendo a formatação original ---
texto_html = """
<div style="background-color: #FFFFFF; color: #333333; padding: 15px; 
            border-left: 5px solid #FFA500; font-family: Arial, sans-serif; 
            text-align: justify; font-size: 16px; line-height: 1.6;">
"""
for palavra in texto.split():
    texto_html += palavra + " "
    output.value = texto_html + "</div>"  
    time.sleep(0.10)  # Esperar 0.1 segundos antes de mostrar a próxima palavra  

# --- Garantir que o texto completo é exibido no final ---
output.value = texto_html + "</div>"

HTML(value='<div></div>')

In [ ]:
from IPython.display import Javascript, display
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Carregar ficheiros .csv e coordenadas ---
def carregar_dados(file_paths):
    dados = {}
    for nome, caminho in file_paths.items():
        if not os.path.exists(caminho):
            raise FileNotFoundError(f"O ficheiro '{caminho}' não foi encontrado para o conjunto '{nome}'")
        print(f"[INFO] A carregar: {nome}")
        try:
            tabela = pd.read_csv(caminho)
        except Exception as e:
            raise ValueError(f"Erro ao carregar o ficheiro '{caminho}': {e}")
        if 'geometry' in tabela.columns:
            tabela['geometry'] = tabela['geometry'].apply(lambda x: wkt.loads(x) if pd.notnull(x) else None)
            tabela = gpd.GeoDataFrame(tabela, geometry='geometry', crs='EPSG:4326')
        elif 'stop_lat' in tabela.columns and 'stop_lon' in tabela.columns:
            tabela['geometry'] = tabela.apply(
                lambda linha: Point(linha['stop_lon'], linha['stop_lat']) 
                if pd.notnull(linha['stop_lon']) and pd.notnull(linha['stop_lat']) else None,
                axis=1
            )
            tabela = gpd.GeoDataFrame(tabela, geometry='geometry', crs='EPSG:4326')
        tabela.dropna(subset=['geometry'], inplace=True)
        dados[nome] = tabela
    return dados

# --- Preparar tabelas finais para análise, juntar os dados e limpar valores em falta ---
def preparar_dados(dados):
    for chave, tabela in dados.items():
        if 'osm_id' not in tabela.columns:
            raise KeyError(f"Coluna 'osm_id' não existe no ficheiro '{chave}'")
        dados[chave]['osm_id'] = dados[chave]['osm_id'].astype(str)
    juncao = [
        {'data': 'categoria_servicos', 'columns': None},
        {'data': 'pop_64_plus', 'columns': None}
    ]
    tabela_junta = dados['distancia_servicos'].copy()
    for bloco in juncao:
        nome_ficheiro = bloco['data']
        colunas_a_juntar = bloco['columns']
        if nome_ficheiro in dados:
            try:
                if colunas_a_juntar is None:
                    colunas_a_juntar = dados[nome_ficheiro].columns
                tabela_junta = pd.merge(tabela_junta, dados[nome_ficheiro][colunas_a_juntar],
                                     on='osm_id', how='left')
                print(f"[INFO] Junção com '{nome_ficheiro}' feita. Total linhas: {tabela_junta.shape[0]}")
            except KeyError as e:
                raise KeyError(f"Erro na junção com '{nome_ficheiro}': {e}.")
        else:
            print(f"[ATENÇÃO] Ficheiro '{nome_ficheiro}' não encontrado. Junção ignorada.")
    colunas_obrigatorias = ['distancia_media_servicos', 'numero_servicos_proximos', 'pop_64_mais']
    tabela_junta.dropna(subset=colunas_obrigatorias, inplace=True)
    variaveis_usar = [
        'numero_servicos_proximos', 'pop_64_mais', 'distancia_media_servicos', 
        'Centro Saude', 'Farmacias', 'Hospitais', 'Supermercados', 
        'Bancos', 'Parques e jardins', 'CTT', 'geometry'
    ]
    for coluna in variaveis_usar:
        if coluna not in tabela_junta.columns:
            tabela_junta[coluna] = None
    tabela_final = tabela_junta[variaveis_usar].copy()
    return tabela_final

# --- Classe para suprimir mensagens durante o carregamento de dados ---
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# --- Caminhos para os ficheiros .csv ---
file_paths = {
    'distancia_servicos': 'distancia_servicos.csv',
    'categoria_servicos': 'categoria_servicos.csv',
    'pop_64_plus': 'pop+64.csv'
}

with SuppressOutput():
    dados = carregar_dados(file_paths)
with SuppressOutput():
    df_servicos = preparar_dados(dados)

colunas_finais = [
    'numero_servicos_proximos', 'pop_64_mais', 'distancia_media_servicos',
    'Centro Saude', 'Farmacias', 'Hospitais', 'Supermercados', 'Bancos',
    'Parques e jardins', 'CTT', 'geometry'
]
df_servicos_final = df_servicos[colunas_finais]

# --- Funções para gráficos e mapas interativos ---
def create_corr_map(gdf):
    colunas = [
        'numero_servicos_proximos', 'pop_64_mais', 'distancia_media_servicos',
        'Centro Saude', 'Farmacias', 'Hospitais', 'Supermercados',
        'Bancos', 'Parques e jardins', 'CTT'
    ]
    colunas = [c for c in colunas if c in gdf.columns]
    matriz = gdf[colunas].corr()
    valores = matriz.values
    heatmap = go.Figure(data=go.Heatmap(
        z=valores,
        x=colunas,
        y=colunas,
        colorscale='Viridis',
        zmin=-1, zmax=1,
        colorbar=dict(title='Correlação', tickfont=dict(color='white')),
        text=np.round(valores, 2),
        texttemplate='%{text}',
        textfont=dict(color='white', size=12),
    ))
    heatmap.update_layout(
        title_font=dict(size=18, color='white'),
        paper_bgcolor='black',
        plot_bgcolor='black',
        font=dict(color='white'),
        xaxis=dict(title='Variáveis', tickangle=45, tickfont=dict(color='white'), side='bottom'),
        yaxis=dict(title='Variáveis', tickfont=dict(color='white'), autorange='reversed'),
        margin=dict(l=100, r=100, t=80, b=150)
    )
    return html.Div([
        html.H3("Correlação entre variáveis", style={'color': 'white', 'text-align': 'center', 'margin-bottom': '20px'}),
        dcc.Graph(figure=heatmap, config={'displayModeBar': False})
    ])

# --- Função para tooltip com total e categorias ---
def extract_service_types(linha):
    try:
        categorias = json.loads(linha.replace("'", "\""))
        total = sum(categorias.values())
        detalhes = '<br>'.join([f"{key} ({value})" for key, value in categorias.items()])
        return f"Total serviços: {total}<br>{detalhes}"
    except (json.JSONDecodeError, AttributeError):
        return 'Total serviços: 0'

def format_decimal_column(tabela, coluna):
    tabela[coluna] = tabela[coluna].apply(lambda x: round(x, 1) if pd.notnull(x) else x)
    return tabela

def create_map_with_services(gdf, coluna_servicos):
    centro = [41.1490, -8.6291]
    mapa = folium.Map(location=centro, zoom_start=13, tiles="CartoDB positron", control_scale=True)
    folium.GeoJson(
        gdf,
        style_function=lambda feature: {'fillColor': 'blue', 'color': 'black', 'weight': 0.5, 'fillOpacity': 0.7},
        tooltip=folium.GeoJsonTooltip(fields=[coluna_servicos], aliases=['Serviços disponíveis: '], sticky=True, parse_html=True)
    ).add_to(mapa)
    mouse_position = MousePosition(
        position='topleft',
        separator=' | ',
        prefix='',
        lat_formatter="function(num) {return L.Util.formatNum(num, 2) + '° N';}",
        lng_formatter="function(num) {return L.Util.formatNum(Math.abs(num), 2) + '° O';}"
    )
    mapa.add_child(mouse_position)
    minimap = MiniMap(toggle_display=True, position='topleft', width=140, height=140, zoom_level_offset=-6, tile_layer="OpenStreetMap")
    mapa.add_child(minimap)
    mapa.get_root().html.add_child(folium.Element("""
    <style>
      .leaflet-control-coordinate {
        top: 10px !important;
        left: 10px !important;
        z-index: 10001 !important;
      }
      .leaflet-control-minimap {
        top: 70px !important;
        left: 10px !important;
        bottom: auto !important;
        z-index: 9999 !important;
      }
    </style>
    """))
    return mapa._repr_html_()

def create_map(gdf, coluna_valor, escala_cores, legenda, nome_tooltip):
    gdf = gdf.dropna(subset=[coluna_valor])
    mapa = folium.Map(location=[41.1490, -8.6291], zoom_start=13, min_zoom=13, tiles="CartoDB positron", control_scale=False)
    mapa.get_root().html.add_child(folium.Element(f"""
    <script>
      L.control.scale({{position: 'topleft', metric: true, imperial: false}}).addTo({mapa.get_name()});
    </script>
    """))
    mapa.get_root().html.add_child(folium.Element("""
    <style>
      .leaflet-control-scale.leaflet-control { top: 60px !important; left: 10px !important; }
      .leaflet-control-coordinate { top: 10px !important; left: 10px !important; background-color: white; padding: 2px 6px; font-size: 12px; font-family: Arial, sans-serif; border-radius: 4px; box-shadow: 0 0 4px rgba(0,0,0,0.2); z-index: 10001 !important;}
      .leaflet-control-minimap { top: 70px !important; left: 10px !important; bottom: auto !important; z-index: 9999 !important;}
    </style>
    """))
    mouse_position = MousePosition(
        position='topleft',
        separator=' | ',
        prefix='',
        lat_formatter="function(num) {return L.Util.formatNum(num, 2) + '° N';}",
        lng_formatter="function(num) {return L.Util.formatNum(Math.abs(num), 2) + '° O';}"
    )
    mapa.add_child(mouse_position)
    minimap = MiniMap(toggle_display=True, position='topleft', width=140, height=140, zoom_level_offset=-6, tile_layer="OpenStreetMap")
    mapa.add_child(minimap)
    escala = escala_cores.scale(gdf[coluna_valor].min(), gdf[coluna_valor].max())
    escala.caption = legenda
    escala.add_to(mapa)
    folium.GeoJson(
        gdf,
        style_function=lambda feat: {
            'fillColor': escala(feat['properties'][coluna_valor]),
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        },
        tooltip=folium.GeoJsonTooltip(fields=[coluna_valor], aliases=[nome_tooltip], sticky=True)
    ).add_to(mapa)
    return mapa._repr_html_()

def mostrar_mapa(gdf, coluna_valor, escala_cores, legenda, nome_tooltip):
    mapa_html = create_map(gdf, coluna_valor, escala_cores, legenda, nome_tooltip)
    return HTML(f'<div style="margin-bottom:-150px;">{mapa_html}</div>')

# --- Ler limites das freguesias e preparar o mapa correspondente ---
BOUNDARY_COLOR   = '#FF8C00'
BOUNDARY_WEIGHT  = 1
BOUNDARY_OPACITY = 1.0

df_freg = pd.read_csv('limites_freguesias_porto.csv', encoding='utf-8')
df_freg['geometry'] = df_freg['geometry'].apply(wkt.loads)
with fiona.open('Continente_CAOP2024.gpkg', layer='cont_municipios') as src:
    original_crs = src.crs
gdf_freg = gpd.GeoDataFrame(df_freg, geometry='geometry', crs=original_crs)
gdf_freg = gdf_freg.to_crs(epsg=4326)
gdf_freg['geometry'] = gdf_freg['geometry'].buffer(0)
minx, miny, maxx, maxy = gdf_freg.total_bounds
center_lat = (miny + maxy) / 2
center_lon = (minx + maxx) / 2

def create_freguesias_map(gdf_freg):
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles=None,
        zoom_control=False,
        scrollWheelZoom=True
    )
    folium.TileLayer(
        tiles='CartoDB.DarkMatter',
        attr='©CartoDB',
        name='CartoDB Dark Matter',
        control=False
    ).add_to(m)
    folium.GeoJson(
        gdf_freg,
        style_function=lambda feat: {
            'fill': True,
            'fillColor': '#00000000',  
            'color':   BOUNDARY_COLOR,
            'weight':  BOUNDARY_WEIGHT,
            'opacity': BOUNDARY_OPACITY
        },
        control=False,
        tooltip=folium.GeoJsonTooltip(
            fields=['freguesia'],
            aliases=['Freguesia: '],
            sticky=True
        )
    ).add_to(m)
    mouse_position = MousePosition(
        position='topleft',
        separator=' | ',
        prefix='',
        lat_formatter="function(num) {return L.Util.formatNum(num, 2) + '° N';}",
        lng_formatter="function(num) {return L.Util.formatNum(Math.abs(num), 2) + '° O';}"
    )
    m.add_child(mouse_position)
    minimap = MiniMap(
        toggle_display=True,
        position='topleft',
        width=140,
        height=140,
        zoom_level_offset=-6,
        tile_layer="OpenStreetMap"
    )
    m.add_child(minimap)
    m.get_root().html.add_child(folium.Element("""
    <style>
      .leaflet-control-coordinate {
        top: 10px !important;
        left: 10px !important;
        z-index: 10001 !important;
      }
      .leaflet-control-minimap {
        top: 70px !important;
        left: 10px !important;
        bottom: auto !important;
        z-index: 9999 !important;
      }
    </style>
    """))
    css = """
    <style>
        .leaflet-control-attribution,
        .leaflet-control-layers {
            display: none !important;
        }
    </style>
    """
    m.get_root().header.add_child(Element(css), name='hide_ui')
    return m._repr_html_()

# --- Definir a estrutura do painel de controlo ---
app = dash.Dash(__name__)

app.index_string = '''
<!DOCTYPE html>
<html>
<head>
    <title>Dashboard</title>
    <style>
        body, html {
            margin: 0;
            padding: 0;
            width: 100%;
            height: 100%;
            background-color: #000;
            overflow-x: hidden;
            overflow-y: auto;
        }
        iframe {
            border: none;
        }
    </style>
</head>
<body>
    {%app_entry%}
    <footer>
        {%config%}
        {%scripts%}
        {%renderer%}
    </footer>
</body>
</html>
'''

app.layout = html.Div([
    html.H1(
        "Dados a utilizar no estudo de caso", 
        style={
            'text-align': 'center', 
            'color': 'white',
            'background-color': '#000', 
            'border': '1px solid white', 
            'padding': '10px',
            'font-weight': 'bold',
            'font-size': '32px'
        }
    ),
    dcc.Tabs(
        id="tabs-maps", 
        value='distancia_servicos', 
        children=[
            dcc.Tab(label='Distância média até aos serviços', value='distancia_servicos',
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'}),
            dcc.Tab(label='Número de serviços próximos', value='numero_servicos_proximos',
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'}),
            dcc.Tab(label='Serviços disponíveis por categoria', value='categoria_servicos',
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'}),
            dcc.Tab(label='População com 65 anos ou +', value='pop_65_plus',
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'}),
            dcc.Tab(label='Matriz de correlação', value='correlacao',
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'}),
            dcc.Tab(label='Freguesias', value='freguesias',
                    style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px'},
                    selected_style={'backgroundColor': '#000', 'color': 'white', 'padding': '10px', 'borderTop': '4px solid #ffcc00'}),
        ],
        style={'backgroundColor': '#000', 'border': '1px solid #333'}
    ),
    dcc.Loading(
        id='loading-mapa',
        type='default',
        color='#007BFF',  # azul
        children=html.Div(id='map-container')
    )
], style={
    'backgroundColor': '#000', 
    'color': 'white', 
    'minHeight': '100vh',
    'margin': '0',
    'padding': '0',
    'overflow': 'auto'
})

# --- Apresentar o mapa correspondente com base no separador selecionado ---
@app.callback(Output('map-container', 'children'), [Input('tabs-maps', 'value')])
def render_map(tab):
    if tab == 'distancia_servicos':
        gdf = format_decimal_column(dados['distancia_servicos'], 'distancia_media_servicos')
        return html.Iframe(
            srcDoc=create_map(
                gdf, 'distancia_media_servicos', linear.YlOrRd_09, 
                'Distância média até aos serviços/por edifício', 'Distância média: '
            ),
            width='100%', height='600'
        )
    elif tab == 'numero_servicos_proximos':
        gdf = dados['categoria_servicos']
        return html.Iframe(
            srcDoc=create_map(
                gdf, 'numero_servicos_proximos', linear.Purples_09, 
                'Número de serviços próximos/por edifício', 'Serviços: '
            ),
            width='100%', height='600'
        )
    elif tab == 'categoria_servicos':
        gdf = dados['categoria_servicos']
        gdf['service_types_text'] = gdf['servicos_por_categoria'].apply(extract_service_types)
        return html.Iframe(
            srcDoc=create_map_with_services(gdf, 'service_types_text'),
            width='100%', height='600'
        )
    elif tab == 'pop_65_plus':  
        gdf = dados['pop_64_plus']
        return html.Iframe(
            srcDoc=create_map(
                gdf, 'pop_64_mais', linear.Reds_09, 
                'População (65 anos ou +/por edifício)', 'População com 65 anos ou mais por edifício: '
            ),
            width='100%', height='600'
        )
    elif tab == 'correlacao':
        return create_corr_map(df_servicos)
    elif tab == 'freguesias':
        return html.Div(
            children=[
                html.Iframe(
                    srcDoc=create_freguesias_map(gdf_freg),
                    width='100%',
                    height='600',
                    style={'border': 'none', 'margin-bottom': '-150px'}
                )
            ]
        )

# --- Guardar ficheiro para uso posterior ---
#df_servicos.to_pickle("data.pkl")

# --- Encontrar uma porta de rede livre para o painel interativo ---
def encontrar_porta_livre():
    while True:
        porta = random.randint(8000, 9000)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("localhost", porta)) != 0:
                return porta

porta = encontrar_porta_livre()

# --- Lançar aplicação interativa quando o ficheiro for executado diretamente ---
if __name__ == '__main__':
    app.run(debug=False, port=porta)

# ---- Mensagem final---
print("\033[92m[INFO] Após análise, pode continuar.\033[0m")

In [ ]:
from IPython.display import Javascript, display
# hide-me
display(Javascript('window.cellVisibilityManager.hideCells();'))

# --- Importar bibliotecas ---
ipython = get_ipython()
ipython.run_line_magic("run", "1.preparacao_bibliotecas.ipynb")

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>

Seguinte: [Análise espacial com agrupamento](3.agrupamento.ipynb)

<div style="border: none; margin: 5px 0; border-top: 1px dashed #FFFFFF; border-bottom: 1px dashed #FFFFFF; height: 5px;"></div>